# 年度业绩报告总体分析

In [17]:
import pandas as pd
import altair as alt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_pickle('../../tidy2022.pkl.gz')

## 数据集

利用`akshare`的业绩报告模块和`tushare`的公司信息，下载全部上市公司自2010年起的业绩报告。

业绩报告包含的信息：

In [3]:
def showProfile(df) :
    col_profile = pd.concat([df.dtypes, df.isna().sum()/len(df)*100, df.sample().T], axis=1 ).reset_index()
    col_profile.columns = ['列名', '类型', '缺失比%','样例']
    
    return (col_profile)


In [4]:
showProfile(df)

,列名,类型,缺失比%,样例
0,股票代码,object,0.000000,603085
1,股票简称,object,0.000000,天成自控
2,每股收益,float64,1.772489,0.08
3,营业收入-营业收入,float64,0.188974,1116288278.28
4,净利润-净利润,float64,0.026506,28594649.9
5,每股净资产,float64,4.812289,3.04879
6,净资产收益率,float64,2.447307,2.6
7,每股经营现金流量,float64,4.056393,0.290983
8,销售毛利率,float64,2.401467,19.524311
9,最新公告日期,object,0.000000,2022-10-31


初步分析此数据集，有如下观点：

- 数据包括了披露业绩的新三板公司。但因为没有其基本信息，所以有些列缺少数据。
- 最后一列时报告期，检视数据时发现2010年的报告期和公告日期有矛盾，但数据集尾部数据正常。
- 净利润和营业收入的缺失值极少，应该不影响大局。
- 净资产收益率缺乏率2.5%，考虑使用。
- 行业缺乏率超过37%，不能用。考虑将新三板的数据去除后使用。

## 上市公司数量分析

- 广义上的上市公司包括了新三板的公司，但一般还是指在北上深交易股票的公司。

In [13]:
cols = df.columns
s_num = df[df.Quarter == 'Q4'].groupby(['Year'])[cols[0]].count()


In [34]:
d_num = s_num.to_frame(name='Num').reset_index()

def plot_hbar(df, x, y, title):

    base = alt.Chart(df).mark_bar().encode(
        x=x,
        y=y
    ).properties(width='container',
                 title=title)

    text = base.mark_text(align='right', dx=-2, color='white').encode(text=x)

    return base+text

plot_hbar(d_num, x='Num:Q', y='Year', title='上市公司数量变化（含新三板）')

alt.LayerChart(...)

- 感觉上，这数据就不太对头了。2022年上市公司总数减少了？
- 查了下新闻，新三板确实有很多摘牌的。但还是很让人怀疑。

In [37]:
sel = (df.Quarter == 'Q4') & (df.market != '其它')
temp = df[sel].groupby(['Year'])[cols[0]].count().to_frame(name='Num').reset_index()
plot_hbar(temp, x='Num:Q', y='Year', title='上市公司数量变化（不含新三板）')

alt.LayerChart(...)

- 从上图看这个数据集有问题。前四年的年报数量居然完全一样，这很难相信。
- 看来需要重新获得数据。 
- 尽管如此，还是可以大致看一下其它数据。

## 上市公司利润分析

In [42]:
df_profit = df[df.Quarter == 'Q4'].groupby(['Year', 'market'])[cols[4]].sum().to_frame(name='Profit').reset_index()

plot_hbar(df_profit, x='sum(Profit)', y='Year', title='Profit move')

alt.LayerChart(...)

In [46]:
def plot_stack_hbar(df, x, y, title, color):

    base = alt.Chart(df).mark_bar().encode(
        x=x,
        y=y,
        color=color
    ).properties(width='container',
                 title=title)

    # text = base.mark_text(align='right', dx=-2, color='white').encode(text=x)

    return base

In [47]:
plot_stack_hbar(df_profit, x='sum(Profit)', y='Year', color = 'market', title='Profit move')

alt.Chart(...)

In [48]:
profit_by_ex = df[df.Quarter == 'Q4'].groupby(['Year', 'exchange'])[cols[4]].sum().to_frame(name='Profit').reset_index()
plot_stack_hbar(profit_by_ex, x='sum(Profit)', y='Year', color = 'exchange', title='Profit move')

alt.Chart(...)

In [51]:
profit_by_area = df[df.Quarter == 'Q4'].groupby(['Year', 'area'])[cols[4]].sum().to_frame(name='Profit').reset_index()
plot_stack_hbar(profit_by_area, x='sum(Profit)', y='Year', color = 'area', title='Profit move').properties(height = 400)

alt.Chart(...)

- 以上结果只是用来留存做为交叉验证使用。
- 需要重新考虑其它途径获得数据。
  - baostock
  - akshare其它源